In [1]:
import os 
%pwd

'/Applications/AI/GAN/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Applications/AI/GAN'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class PrepareGANModelConfig:
    root_dir: Path
    generator_model_path: Path
    discriminator_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_generator_filters: int
    params_discriminator_filters: int
    params_use_bias: bool
    params_dropout_rate: float
    
from src.GAN.constants import *
from src.GAN.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_gan_model_config(self) -> PrepareGANModelConfig:
        config = self.config.prepare_gan_model

        create_directories([config.root_dir])

        prepare_gan_model_config = PrepareGANModelConfig(
            root_dir=Path(config.root_dir),
            generator_model_path=Path(config.generator_model_path),
            discriminator_model_path=Path(config.discriminator_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_generator_filters=self.params.GENERATOR_FILTERS,
            params_discriminator_filters=self.params.DISCRIMINATOR_FILTERS,
            params_use_bias=self.params.USE_BIAS,
            params_dropout_rate=self.params.DROPOUT_RATE
        )

        return prepare_gan_model_config

In [7]:
# GAN Model Preparation: components
import tensorflow as tf

class PrepareGANModel:
    def __init__(self, config: PrepareGANModelConfig):
        self.config = config
    
    def build_generator(self):
        """
        Builds a U-Net style generator for image-to-image translation
        """
        inputs = tf.keras.layers.Input(shape=self.config.params_image_size)
        
        # Encoder (downsampling)
        enc1 = self._downsample(inputs, self.config.params_generator_filters)
        enc2 = self._downsample(enc1, self.config.params_generator_filters * 2)
        enc3 = self._downsample(enc2, self.config.params_generator_filters * 4)
        enc4 = self._downsample(enc3, self.config.params_generator_filters * 8)
        
        # Bottleneck
        bottleneck = self._downsample(enc4, self.config.params_generator_filters * 8)
        
        # Decoder (upsampling with skip connections)
        dec1 = self._upsample(bottleneck, enc4, self.config.params_generator_filters * 8)
        dec2 = self._upsample(dec1, enc3, self.config.params_generator_filters * 4)
        dec3 = self._upsample(dec2, enc2, self.config.params_generator_filters * 2)
        dec4 = self._upsample(dec3, enc1, self.config.params_generator_filters)
        
        # Output layer
        outputs = tf.keras.layers.Conv2D(3, 4, padding='same', activation='tanh')(dec4)
        
        generator = tf.keras.Model(inputs=inputs, outputs=outputs)
        generator.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate, beta_1=0.5),
            loss='mae'
        )
        
        return generator
    
    def build_discriminator(self):
        """
        Builds a PatchGAN discriminator for image classification
        """
        inputs = tf.keras.layers.Input(shape=self.config.params_image_size)
        
        # Layer 1
        x = tf.keras.layers.Conv2D(
            self.config.params_discriminator_filters, 4, strides=2, padding='same',
            use_bias=self.config.params_use_bias)(inputs)
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        
        # Layer 2
        x = tf.keras.layers.Conv2D(
            self.config.params_discriminator_filters * 2, 4, strides=2, padding='same',
            use_bias=self.config.params_use_bias)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        
        # Layer 3
        x = tf.keras.layers.Conv2D(
            self.config.params_discriminator_filters * 4, 4, strides=2, padding='same',
            use_bias=self.config.params_use_bias)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        
        # Layer 4
        x = tf.keras.layers.Conv2D(
            self.config.params_discriminator_filters * 8, 4, strides=1, padding='same',
            use_bias=self.config.params_use_bias)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        
        # Output layer
        outputs = tf.keras.layers.Conv2D(1, 4, strides=1, padding='same')(x)
        
        discriminator = tf.keras.Model(inputs=inputs, outputs=outputs)
        discriminator.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate, beta_1=0.5),
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        
        return discriminator
    
    def _downsample(self, inputs, filters, size=4, strides=2, apply_batchnorm=True):
        """Helper function for downsampling layers in the generator"""
        x = tf.keras.layers.Conv2D(filters, size, strides=strides, padding='same',
                                  use_bias=self.config.params_use_bias)(inputs)
        
        if apply_batchnorm:
            x = tf.keras.layers.BatchNormalization()(x)
            
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        
        return x
    
    def _upsample(self, inputs, skip_connection, filters, size=4, strides=2, apply_dropout=False):
        """Helper function for upsampling layers in the generator"""
        x = tf.keras.layers.Conv2DTranspose(filters, size, strides=strides, padding='same',
                                           use_bias=self.config.params_use_bias)(inputs)
        
        x = tf.keras.layers.BatchNormalization()(x)
        
        if apply_dropout:
            x = tf.keras.layers.Dropout(self.config.params_dropout_rate)(x)
            
        x = tf.keras.layers.ReLU()(x)
        
        # Skip connection (concatenate)
        x = tf.keras.layers.Concatenate()([x, skip_connection])
        
        return x
    
    def prepare_gan_model(self):
        """
        Prepares both generator and discriminator models and saves them
        """
        generator = self.build_generator()
        discriminator = self.build_discriminator()
        
        # Save models
        self.save_model(self.config.generator_model_path, generator)
        self.save_model(self.config.discriminator_model_path, discriminator)
        
        return generator, discriminator
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model) -> None:
        model.save(path)
        
# GAN Model Preparation: pipeline
try:
    config = ConfigurationManager()
    prepare_gan_model_config = config.get_prepare_gan_model_config()
    prepare_gan_model = PrepareGANModel(config=prepare_gan_model_config)
    generator, discriminator = prepare_gan_model.prepare_gan_model()
except Exception as e:
    raise e

/Applications/AI/GAN/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[2025-03-20 09:51:54,675: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-20 09:51:54,677: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-20 09:51:54,678: INFO: common: created directory at: artifacts]
[2025-03-20 09:51:54,679: INFO: common: created directory at: artifacts/prepare_gan_model]
[2025-03-20 09:51:54,918: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
[2025-03-20 09:51:54,921: WARNING: __init__: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.Adam`.]
[2025-03-20 09:51:54,981: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam

/Applications/AI/GAN/.venv/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
